In [23]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import community
import pickle 
import pandas as pd

## Todos

TODO: Add diagnostics showing how long some tasks take to run

TODO: Other community structures and diagnostics as described [here](https://networkx.github.io/documentation/stable/reference/algorithms/community.html)

TODO: Recreate a graph, maybe in GEPHI, using whatever communinty structure we end up with

TODO: LSA / Semantic analysis



## Load Graph
Read in the edge list, which contains all relevant information: source author -> target author, number of times cited

In [120]:
# G = nx.read_weighted_edgelist("./tiny_edge_list.csv", delimiter=",")
G_directed = nx.read_weighted_edgelist("./small_edge_list.csv", delimiter=",",create_using= nx.DiGraph())
G = nx.read_weighted_edgelist("./small_edge_list.csv", delimiter=",")

In [121]:
print(nx.info(G_directed))

Name: 
Type: DiGraph
Number of nodes: 4369
Number of edges: 41158
Average in degree:   9.4205
Average out degree:   9.4205


## Various node orderings

In [122]:
# Is it weighted?
degreeTable = pd.DataFrame(G_directed.in_degree, columns = ["Name","In Degree"])
sd = degreeTable.sort_values(by=['In Degree'], ascending=False)
sd.head(10)

,Name,In Degree
59,HUSSERL E,1108
224,HEIDEGGER M,795
1186,MERLEAUPONTY M,656
888,KANT I,363
2011,SARTRE J,298
2072,DERRIDA J,294
221,ZAHAVI D,285
2207,LEVINAS E,272
706,RICOEUR P,262
2413,HEGEL G,260


In [123]:
degreeTable = pd.DataFrame(G_directed.out_degree, columns = ["Name","Out Degree"])
sd = degreeTable.sort_values(by=['Out Degree'], ascending=False)
sd.head(10)

,Name,Out Degree
23,MORAN D,228
221,ZAHAVI D,187
1265,FUCHS T,167
423,GALLAGHER S,149
417,FROESE T,139
162,HEINAMAA S,129
1016,SCHWITZGEBEL E,116
7,DEPRAZ N,111
118,WALSH P,110
1088,KRIEGEL U,110


In [126]:
# Not sure if results differ between G and G_directed...
h,a=nx.hits(G)

In [127]:
authTable =  pd.DataFrame.from_dict(a,orient='index', columns=["Authority"])
sd = authTable.sort_values(by=['Authority'], ascending=False)
sd.head(20)

,Authority
HUSSERL E,0.020710
ZAHAVI D,0.012081
HEIDEGGER M,0.008453
MERLEAUPONTY M,0.007427
DEPRAZ N,0.006937
MORAN D,0.006112
CROWELL S,0.005290
HEINAMAA S,0.004736
GALLAGHER S,0.004640
WALDENFELS B,0.003687


In [128]:
hubTable =  pd.DataFrame.from_dict(h,orient='index', columns=["Hubitude"])
sd = hubTable.sort_values(by=['Hubitude'], ascending=False)
sd.head(20)

,Hubitude
HUSSERL E,0.020710
ZAHAVI D,0.012081
HEIDEGGER M,0.008453
MERLEAUPONTY M,0.007427
DEPRAZ N,0.006937
MORAN D,0.006112
CROWELL S,0.005290
HEINAMAA S,0.004736
GALLAGHER S,0.004640
WALDENFELS B,0.003687


## Community Detection

See https://networkx.github.io/documentation/stable/reference/algorithms/community.html

Currently using girvan_newman but it is slow, and only uses a single processor.

In [167]:
from itertools import tee

In [171]:
# Print out the number of members of each communinty and sub-community.  
# Note that results is a generator, so after running this once the generator is "unwound" and must be recreated
def conmunity_stats(results, level = 2):
    for l1 in results:
        print("Level 1", len(l1))
        if level > 1:
            for l2 in l1:
                print("Level 2", len(l2))

#### Girvan Newman

Takes a long time...

In [24]:
communities_generator = community.girvan_newman(G)

In [16]:
top_level_communities = next(communities_generator)

In [19]:
next_level_communities = next(communities_generator)

In [2]:
# sorted(map(sorted, next_level_communities))

In [42]:
# Save communites
pickle.dump(next_level_communities, open("gn_communities.obj", "wb"))

In [175]:
# Testing re-open
gn = pickle.load(open("gn_communities.obj", "rb"))

In [178]:
conmunity_stats(gn,1)

Level 1 4364
Level 1 3
Level 1 2


### K Clique Communities

In [172]:
cg2 = community.k_clique_communities(G,6)

In [174]:
# conmunity_stats(cg2,1)

## Draw graph

In [ ]:
# nx.draw(G, node_size = 10, with_labels=True, font_size = 12)